# Cryptocurrency Fundamental Analysis Tool

In [ ]:
import json
import requests
import pandas as pd
import hvplot.pandas
import panel as pn
import holoviews as hv
import numpy as np
from pathlib import Path


In [144]:

# Coin Metrics API endpoint url
url = 'https://community-api.coinmetrics.io/v4/timeseries/asset-metrics?'

# Dates to get historical data for the last 3 years
start_date = pd.Timestamp("2018-01-01").isoformat()
end_date = pd.Timestamp.today().isoformat()

# Number of records to retrieve
page_size = 6000

# Define the list of crypto assets to analyze - Unable to loop through CSV file
crypto_asset = ['btc', 'eth', 'doge']

def get_data(metric):
    
    # Save the JSON response as lists
    symbol = []
    time = []
    value = []

    # Loop through the selected tokens
    for token in crypto_asset:
        
        # Build URL with the corresponding token, metric, and dates
        get_url = f"{url}&assets={token}&metrics={metric}&start_time={start_date}&end_time={end_date}&page_size={page_size}"

        # Get the response from Coin Metrics API is JSON format
        response = requests.get(get_url).json()

        # Iterate through the response to add the relevant items to its corresponding list
        for item in response['data']:

            symbol.append(item['asset'])
            time.append(item['time'])
            value.append(item[metric])
            
    # Build the dataframe with the data
    data_df = pd.DataFrame(
    {'Token': symbol,
     'Date': time,
     metric: value
    })
    
    # Format the date for easy reading 
    data_df['Date'] = pd.to_datetime(data_df['Date'], format="%Y/%m/%d").dt.date
    
    # Set Date as the index
    data_df.index = data_df['Date']
    
    # Change the value type to 'float' to allow graphing
    data_df[metric] = data_df[metric].astype(float)
    
    # Define columns
    data_df = data_df.drop(columns='Date')
    data_df.columns = ['Token', metric]
    
    # Format the DataFrame to allow xgraphing
    #updated_data_df = data_df.pivot_table('Value', ['Date'], 'Token')
    
    #return updated_data_df
    
    return data_df
    

In [145]:
# Parse metrics
active_addresses = get_data('AdrActCnt')
transaction_count = get_data('TxCnt')
supply = get_data('SplyCur')
mc = get_data('CapMrktCurUSD')
mc_real = get_data('CapRealUSD')
velocity = get_data('VelCur1yr')

In [146]:
active_addresses[active_addresses['Token'] == crypto_asset[-1]]

,Token,AdrActCnt
Date,,
2018-01-01,doge,66246.0
2018-01-02,doge,77363.0
2018-01-03,doge,74585.0
2018-01-04,doge,86339.0
2018-01-05,doge,90058.0
...,...,...
2021-09-30,doge,50654.0
2021-10-01,doge,54001.0
2021-10-02,doge,53794.0


In [147]:
df = pd.concat([active_addresses, transaction_count, supply, mc, mc_real, velocity], join='outer', axis=1)
df = df.loc[:, ~df.columns.duplicated()]
df

,Token,AdrActCnt,TxCnt,SplyCur,CapMrktCurUSD,CapRealUSD,VelCur1yr
Date,,,,,,,
2018-01-01,btc,972783.0,241625.0,1.677635e+07,2.258877e+11,8.384112e+10,37.984407
2018-01-02,btc,1142721.0,343664.0,1.677849e+07,2.475552e+11,8.462143e+10,38.001934
2018-01-03,btc,1130917.0,396942.0,1.678045e+07,2.518793e+11,8.508670e+10,37.983200
2018-01-04,btc,1253986.0,424762.0,1.678255e+07,2.529181e+11,8.590903e+10,37.965540
2018-01-05,btc,1069525.0,339441.0,1.678434e+07,2.852872e+11,8.745871e+10,37.928875
...,...,...,...,...,...,...,...
2021-09-30,doge,50654.0,16802.0,1.313639e+11,2.676759e+10,1.785329e+10,79.613266
2021-10-01,doge,54001.0,19533.0,1.313773e+11,2.916846e+10,1.785816e+10,79.701791
2021-10-02,doge,53794.0,20237.0,1.313909e+11,2.883334e+10,1.786202e+10,79.880047


In [126]:
# UNABLE TO ITERATE THROUGH THE TOKEN LIST CSV FILE - ERROR WITH COINMETRICS?

# Coin Metrics API endpoint url
url = 'https://community-api.coinmetrics.io/v4/timeseries/asset-metrics?'

# Dates to get historical data for the last 3 years
start_date = pd.Timestamp("2018-01-01").isoformat()
end_date = pd.Timestamp.today().isoformat()

# Number of records to retrieve
page_size = 6000

# Define the list of crypto assets to analyze
crypto_asset = pd.read_csv(Path('./token_list.csv'), header=None)

tokens = token_list.values.tolist()

def get_data(metric):
    
    # Save the JSON response as lists
    symbol = []
    time = []
    value = []

    # Loop through the selected tokens
    for token in tokens:
        
        # Build URL with the corresponding token, metric, and dates
        get_url = f"{url}&assets={token}&metrics={metric}&start_time={start_date}&end_time={end_date}&page_size={page_size}"

        # Get the response from Coin Metrics API is JSON format
        response = requests.get(get_url).json()

        # Iterate through the response to add the relevant items to its corresponding list
        for item in response['data']:

            symbol.append(item['asset'])
            time.append(item['time'])
            value.append(item[metric])
            
    # Build the dataframe with the data
    data_df = pd.DataFrame(
    {'Token': symbol,
     'Date': time,
     metric: value
    })
    
    # Format the date for easy reading 
    data_df['Date'] = pd.to_datetime(data_df['Date'], format="%Y/%m/%d").dt.date
    
    # Set Date as the index
    data_df.index = data_df['Date']
    
    # Change the value type to 'float' to allow graphing
    data_df[metric] = data_df[metric].astype(float)
    
    # Define columns
    data_df = data_df.drop(columns='Date')
    data_df.columns = ['Token', metric]
    
    # Format the DataFrame to allow xgraphing
    #updated_data_df = data_df.pivot_table('Value', ['Date'], 'Token')
    
    #return updated_data_df
    
    return data_df
    

In [127]:
get_data('AdrActCnt')

KeyError: 'data'